# (solution) Supermarket Accessibility in Graz


In [ ]:
from keplergl import KeplerGl
import osmnx as ox
import pandas as pd
import geopandas as gpd

In [ ]:
PLACE_NAME:str = "Graz, Austria"

# Get the boundary polygon of Graz and project it to EPSG:31256
poly_border_graz = ox.geocode_to_gdf(PLACE_NAME).to_crs(epsg=31256)
poly_border_graz.plot()

In [ ]:
supermarket_osm_tags = {"shop": "supermarket"}

# download all supermarkets in Graz
graz_supermarkets = ox.features_from_place(PLACE_NAME, tags=supermarket_osm_tags).to_crs(epsg=31256)
graz_supermarkets.plot()

In [ ]:
# clean the supermarket data and separate points and polygons
graz_supermarkets = graz_supermarkets[['geometry', 'brand']]

graz_supermarkets_points = graz_supermarkets.loc['node']
graz_supermarkets_polygons = graz_supermarkets.loc['way']

# transform polygons to points by using point on surface
graz_supermarkets_polygons = graz_supermarkets_polygons.set_geometry(
    graz_supermarkets_polygons.geometry.representative_point()
)

# merge points and polygons
# Concatenate rows; keep geometry column and CRS explicit
graz_all_supermarkets = gpd.GeoDataFrame(
    pd.concat([graz_supermarkets_points, graz_supermarkets_polygons], ignore_index=True),
    geometry="geometry",
    crs=graz_supermarkets_points.crs
).dropna(subset='brand').reset_index(drop=True)
graz_all_supermarkets.plot()

In [ ]:
# visualize the supermarkets in kepler.gl
k_map = KeplerGl(height=500)
k_map.add_data(data=graz_all_supermarkets.set_geometry(graz_all_supermarkets.geometry), name="Graz supermarket")
k_map

In [ ]:
# extract Billa shops
billa_supermarkets = graz_all_supermarkets[graz_all_supermarkets['brand'].str.contains('spar', case=False, na=False)].reset_index(drop=True)
billa_supermarkets.brand.unique()

In [ ]:
# create buffers around Billa shops and display in Kepler.gl
DISTANCE_STEPS:list[int] = [250,500,1000,2500] # in meters

buffers: dict[int, gpd.GeoDataFrame] = {}

# loop in reversed order to have the largest buffer at the bottom
for dist in DISTANCE_STEPS:
    buf = billa_supermarkets[['geometry']].copy() # create a copy of the original geometry
    # 1. create a buffer of the given distance
    buf['geometry'] = buf.geometry.buffer(dist)
    # 2. dissolve all geometries into one single geometry 
    # 3. clip the buffer to the boundary of Graz 
    buf = buf.dissolve(by=None, as_index=False).clip(poly_border_graz)
    # 4. add buffer to the buffers list with name
    buffers[dist] = buf[['geometry']]


# create a new kepler.gl map
k_map3 = KeplerGl(height=500)
k_map3.add_data(data=billa_supermarkets[['brand','geometry']], name="Billas locations")

for dist in buffers:
    k_map3.add_data(buffers[dist], name=f"Buffer {dist} m")

k_map3.add_data(data=poly_border_graz, name="Graz boundary")

k_map3 

In [ ]:
def area_sum(gdf: gpd.GeoDataFrame) -> float:
    return float(gdf.geometry.area.sum())

BUFFER_MID_DISTANCE:list[int] = [125, 375, 750, 1750]  # mid-points of each ring
OUTSIDE_MID_DISTANCE:int = 3000  # representative distance for areas beyond largest ring
 
AREAN_CITY_FULL:float = area_sum(poly_border_graz) # total area of Graz

# Calculate area of each ring
ring_areas: list[float] = []
for i, dist in enumerate(DISTANCE_STEPS):
    if i == 0:
        ring_areas.append(area_sum(buffers[dist])) # first ring
    else:
        ring_areas.append(
            area_sum(buffers[dist]) - 
            area_sum(buffers[DISTANCE_STEPS[i-1]]) # subtract area of previous (smaller) ring
        )

outside_area = AREAN_CITY_FULL - area_sum(buffers[DISTANCE_STEPS[-1]]) # area outside largest ring

# area-weighted "average distance" (using mid-points) = mean
avg_dist_m = (sum(a*m for a, m in zip(ring_areas, BUFFER_MID_DISTANCE)) + outside_area*OUTSIDE_MID_DISTANCE) / AREAN_CITY_FULL
print(f"Pseudo-average distance to nearest Billa (city-wide): {avg_dist_m:.0f} m")

#### 🥊 Challenge

Try to do the same for each district in Graz. Which district has "more access" to supermarkets?  

In [ ]:
# this line is to clear the output of the notebook, so that when you commit it, it is clean
!jupyter nbconvert --clear-output --inplace lab_01_ex.ipynb